# < 모델링 >

#### Train


1. 미주: america_train.pkl
2. 유럽: europe_train.pkl
3. 중국 외 아시아: asia_train.pkl


#### Test

1. 미주: america_test.pkl
2. 유럽: europe_test.pkl
3. 중국 외 아시아: asia_test.pkl


<br>

#### 활용 모델


1. XGBoosting( x )
2. GradientBoosting( x )
3. RandomForest( o )


# 1. Train, Test 분리: 미주, 유럽, 중국 외 아시아

In [138]:
# Data Load
# 해외유입 누적 확진자 수(총 1,118명)에서 약 3개월 간 중국(19명), 아프리카(3명), 호주(0명)으로
# 해당 대륙에 대한 예측에 다소 무리가 있고, 전체 해외유입에 영향력이 크지 않다고 판단.
# 미주, 유럽, 중국 외 아시아에 대해서 적합한 모델을 만든 후, 결괏값을 합산하여 최종 submission을 구성.


# 미주
america_train = pd.read_pickle("./data_processed/Z.Train_test/america_train.pkl")
america_test = pd.read_pickle("./data_processed/Z.Train_test/america_test.pkl")

# 유럽
europe_train = pd.read_pickle("./data_processed/Z.Train_test/europe_train.pkl")
europe_test = pd.read_pickle("./data_processed/Z.Train_test/europe_test.pkl")

# 중국 외 아시아
asia_train = pd.read_pickle("./data_processed/Z.Train_test/asia_train.pkl")
# NaN 값 제거 후 2월 4일부터 사용.
asia_train = asia_train.dropna()
asia_test = pd.read_pickle("./data_processed/Z.Train_test/asia_test.pkl")

In [139]:
# 데이터 크기 확인.
print(america_train.shape)
print(america_test.shape)
print(europe_train.shape)
print(europe_test.shape)
print(asia_train.shape)
print(asia_test.shape)

(58, 171)
(14, 170)
(70, 171)
(14, 170)
(92, 171)
(14, 170)


In [140]:
######## train 데이터에 대한 수가 적기 때문에 train을 train set, validation set으로 나누는 것은 부적합.
# 5월 5일까지의 데이터를 train으로, 5월 6일 ~ 5월 10일까지의 target 값을 test로 하여 가장 잘 맞는 모델을 선택.

# X_train set: 미주, 유럽, 중국 외 아시아
X_train_america = america_train.drop(['Date', 'America_유입'], axis=1)
X_train_europe = europe_train.drop(['Date', 'Europe_유입'], axis=1)
X_train_asia = asia_train.drop(['Date', 'Asia_out_china_유입'], axis=1)

# y_train set: 미주, 유럽, 중국 외 아시아
y_train_america = america_train['America_유입']
y_train_europe = europe_train['Europe_유입']
y_train_asia = asia_train['Asia_out_china_유입']

# X_test set(5월6일~5월19일까지의 input data): 미주, 유럽, 중국 외 아시아.
# 5월 6일부터 5월 19까지 target 값을 예측하는데 사용.

X_test_america = america_test.drop('Date', axis=1)
X_test_europe = europe_test.drop('Date', axis=1)
X_test_asia = asia_test.drop('Date', axis=1)

# 2. 활용할 함수 정의

In [141]:
# metric 정의
# 포스트 코로나 챌린지 평가 함수.

def post_corona_metric(data_frame):
    import numpy as np
    
    prediction = data_frame['pred'].values
    actual = data_frame['actual'].values
    
    diff = np.square(prediction - actual).sum()
    
    over = np.square(actual).sum()
    
    return 100 * (1 - diff/over)

In [142]:
# trend를 반영한 확진자 예측.
# Hodrick Prescott Filter 활용.

def get_target_series(target_series, lamb=1600):
    
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    import math
    
    # target에 대한 
    cycle, trend = sm.tsa.filters.hpfilter(target_series, lamb=lamb)

    target_df = pd.DataFrame(target_series)
    target_df.columns = ['target']
    target_df["cycle"] = cycle
    target_df["trend"] = trend
    
    # trend값이 음수인 것은 0으로 치환.
    target_df[target_df['trend'] <= 0] = 0
    
    return target_df['trend'].round() # '기존 target 값', 'cycle', 'trend' / 기존 target 대신 trend 값을 활용.

In [143]:
# 타겟 데이터.
# target 값의 trend를 새로운 target 값으로 지정.

y_train_america_trend = get_target_series(y_train_america)
y_train_europe_trend = get_target_series(y_train_europe)
y_train_asia_trend = get_target_series(y_train_asia)

# 3. Prediction

In [144]:
# X_train: 각 대륙별 데이터. target 제외한 나머지 컬럼.
# y_train: X_train에 대한 target 값.
# X_test: 5월 6일 ~ 19일까지의 input data.

# 각 대륙별 유입 확진자의 'trend'를 'target'으로 학습.

def rf_model(X_train, y_train, X_test):

    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=6)

    forest = RandomForestRegressor(n_estimators=6, random_state=1, bootstrap=True)
    forest.fit(X_train, y_train)

    print("훈련 세트 정확도: {:.3f}".format(forest.score(X_train, y_train)))
    print("검증 세트 정확도: {:.3f}".format(forest.score(X_valid, y_valid)))

    result_valid = pd.DataFrame({'pred':forest.predict(X_valid).round(), 'actual':y_valid})

    print("metric score: '{:.2f}'".format(post_corona_metric(result_valid)))
    
    # 5월 6일 ~ 5월 19일 데이터로 뽑은 prediction 값.
    result = pd.Series(forest.predict(X_test)).round()

    return result

# 4. 평가

In [147]:
result_america = rf_model(X_train_america, y_train_america_trend, X_test_america)
result_europe = rf_model(X_train_europe, y_train_europe_trend, X_test_europe)
result_asia = rf_model(X_train_asia, y_train_asia_trend, X_test_asia)

# 각 데이터에 대한 결과.
each_result = pd.DataFrame({'america': result_america, 'europe': result_europe, 'asia': result_asia})
each_result.index = pd.date_range('2020-05-06', periods=14)

# 최종 결과
pred_result = result_america + result_europe + result_asia
pred_result.index = pd.date_range('2020-05-06', periods=14)

훈련 세트 정확도: 0.961
검증 세트 정확도: 0.885
metric score: '94.63'
훈련 세트 정확도: 0.962
검증 세트 정확도: 0.869
metric score: '95.27'


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33653)
Traceback (most recent call last):
  File "/dsvm/tools/spark/current/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/dsvm/tools/spark/current/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33653)
Traceback (most recent call last):
  File "/dsvm/tools/spark/current/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above e

훈련 세트 정확도: 0.969
검증 세트 정확도: 0.833
metric score: '92.16'


In [148]:
# 5월 6일부터 5월 19일까지 각 일자별 해외 유입 확진자 예측값('sum' column)

each_result.merge(pd.DataFrame(pred_result, columns=['sum']), left_index=True, right_index=True)

,america,europe,asia,sum
2020-05-06,4.0,2.0,1.0,7.0
2020-05-07,5.0,2.0,1.0,8.0
2020-05-08,4.0,2.0,2.0,8.0
2020-05-09,3.0,1.0,1.0,5.0
2020-05-10,3.0,1.0,1.0,5.0
2020-05-11,3.0,1.0,1.0,5.0
2020-05-12,3.0,1.0,1.0,5.0
2020-05-13,3.0,1.0,2.0,6.0
2020-05-14,2.0,1.0,2.0,5.0
2020-05-15,3.0,1.0,2.0,6.0
